In [0]:
#Parametros para facilitar a execução e reutilização do notebook para carga de diferentes anos e estados
dbutils.widgets.text("anos_param", "2023,2022", "Lista de Anos")
dbutils.widgets.text("uf", "MG", "UF")

In [0]:
#Variaveis do Unity Catalog
CATALOGO = "agroclima"
SCHEMA = "bronze"
TABELA = "producao_milho"

# Montar caminho da tabela
caminho_tabela = f"{CATALOGO}.{SCHEMA}.{TABELA}"



In [0]:
# Transforma os parametros em variaveis
lista_anos = dbutils.widgets.get("anos_param").split(",")
uf = dbutils.widgets.get("uf")

In [0]:
# imports

import json
from utils.api import buscar_dados
from pyspark.sql.functions import current_timestamp
 

In [0]:
# Função que Busca o Id da UF informada no parametro
def obter_estado_id_por_uf(uf):
    IBGE_ESTADO_URL =  f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/{uf}/"
    dados = buscar_dados(IBGE_ESTADO_URL)
    estado_id = dados['id']
    return estado_id

In [0]:
# Seta Variavel com o Id da UF 
estado_id = obter_estado_id_por_uf(uf)


In [0]:
# Função que consome a API do IBGE e retorna dados de produção de milho
def obter_producao_milho(ano, estado_id):
       
    # Cria URL com base nos parametros passados
    IBGE_URL = f"https://servicodados.ibge.gov.br/api/v3/agregados/839/periodos/{ano}/variaveis/109|216|214?localidades=N6[N3[{estado_id}]]&classificacao=81[114253,114254]"
    
    # Executa API do IBGE buscando dados da Produção de Milho
    dados_producao_milho = buscar_dados(IBGE_URL)

    #Obs: Devido a complexidade na estrutura do json, ele esta sendo transformado em String para manter o dados 
    # originais e evitar realizar qualuqer tipo de transformação na camada Bronze

    #Transforma Json em String para ser armazenado no dataframe
    string_dados = json.dumps(dados_producao_milho)
    
    #Cria Dataframe com dados do Json
    df = spark.createDataFrame([(string_dados,)], ["string_dados"])

    #Adiciona colunas para controle de carga
    df = df.withColumn("ano", lit(ano)) \
           .withColumn("uf", lit(uf)) \
           .withColumn("_data_ingestao", current_timestamp())

    return df


In [0]:
for ano in lista_anos:
    try:
        # Chama Função que consome a API com cada paramtero de Ano da Lista passada em parametro
        df = obter_producao_milho(ano, estado_id)

        #Cria Tabela Delta no Unity Catalog  atualizando somente ano e uf passado em parametro
        df.write.format("delta") \
            .mode("overwrite") \
            .option("replaceWhere", f"ano = {ano} AND uf = '{uf}'") \
            .option("mergeSchema", "true") \
            .saveAsTable(caminho_tabela)

        print(f"Ano {ano} salvo com sucesso.")

    except Exception as e:
        print(f"Erro ao processar ano {ano}: {str(e)}")